In [2]:
import pandas as pd
import lyricsgenius as lg
from bs4 import BeautifulSoup
import numpy as np
import re
import nltk
from collections import Counter

After completing putting together a dataset of 10 thousand songs across 5 genres containing data regarding each songs musical attributes as measured by Spotify, we proceed to find a way to add the lyrics of each song to our dataset. I suspected that Lyrics Genius, one of the most popular music lyrics websites which also seeks to explain the history and meaning of lyrics via user submitted explanations, would be a great place to start since Genius already has an embedded application in Spotify through their front end interface where certain songs allow for viewing lyrics and song history that's brought by Genius. 

Upon doing some research, I discovered that, while Spotify doesn't directly house Genius lyrics directly in their database, their exists a Python wrapper which enables API calls to Genius through a package called 'lyricsgenius'. I downloaded this package and discovered that it works similar to the SpotiPy package in that it only requires authentication by registering with Genius as a developer and optaining a token and client secret. I proceed to generate these after registering and stored them in variables.

In [2]:
#Store token code and instatiate API call
token = '1Y2rboTmLHNoR2NjoLrPXK0z0Oes_BiHf9gYT6bjTm_G_vXlzg7bv19GSOzGNQS2'
genius = lg.Genius(token)

In [3]:
#Read in dataframe from Spotify Data Extraction section containing musical attributes
spotify_df = pd.read_csv('spotify_genre_df.csv')

With the authentication and token code ready to go, we can proceed to utilize the API wrapper to extract the lyrics from the Genius website. The Python wrapper appears to work by searching the website using the song and artist named input into the "search_song" function and returning a scraping of the HTML retrieved via BeautifulSoup, with the output including only the lyrics body and song structure labels but not the other text such as comments and annotated meanings. 

As such, we will use this function to iterate through all the songs in our existing Spotify attibute dataframe and attempt to pull the lyrics into the dataframe by searching the values in the track name and artist field, while placing a placeholder of 'Lyrics not found' if the call fails and generates an error. We will additionally add a column to the dataframe for lyrics and fill it with 'insert lyrics' in order to identify cells which have not had lyrics retrieved yet. The calls take an extensive amount of time to execute (approximately 60 minutes per 2000 songs) so we will run 5 separate queries to pull data for each genre and save off the file in order prevent the API call from potentially hanging and to keep track of what's already been retrieved.

In [48]:
#Update Spotify attributes dataframe with lyrics column - this will be populated through the LyricsGenius API
spotify_df['lyrics'] = '[insert lyrics]'

In [5]:
#Turn off output messaging when retrieving lyrics to reduce processing and unnecessary output and improve speed
genius.verbose = False

In [ ]:
#Iterate through each row of the Spotify dataframe to search for lyrics based on the 'track_name' and 'artist' column
#Some songs may not find match to a lyrics page in the Genius website, in which case the except will populate message
for i in spotify_df[spotify_df['playlist_name'] == 'Rock'].index:
    try:
        spotify_df['lyrics'][i] = genius.search_song(spotify_df['track_name'][i], spotify_df['artist'][i]).lyrics
    except: 
        spotify_df['lyrics'][i] = '[Lyrics not found]'
        pass

In [ ]:
#Iterate through each row of the Spotify dataframe to search for lyrics based on the 'track_name' and 'artist' column
#Some songs may not find match to a lyrics page in the Genius website, in which case the except will populate message
for i in spotify_df[spotify_df['playlist_name'] == 'Pop'].index:
    try:
        spotify_df['lyrics'][i] = genius.search_song(spotify_df['track_name'][i], spotify_df['artist'][i]).lyrics
    except: 
        spotify_df['lyrics'][i] = '[Lyrics not found]'
        pass

In [ ]:
#Iterate through each row of the Spotify dataframe to search for lyrics based on the 'track_name' and 'artist' column
#Some songs may not find match to a lyrics page in the Genius website, in which case the except will populate message
for i in spotify_df[spotify_df['playlist_name'] == 'Hip-Hop'].index:
    try:
        spotify_df['lyrics'][i] = genius.search_song(spotify_df['track_name'][i], spotify_df['artist'][i]).lyrics
    except: 
        spotify_df['lyrics'][i] = '[Lyrics not found]'
        pass

In [ ]:
#Iterate through each row of the Spotify dataframe to search for lyrics based on the 'track_name' and 'artist' column
#Some songs may not find match to a lyrics page in the Genius website, in which case the except will populate message
for i in spotify_df[spotify_df['playlist_name'] == 'Country'].index:
    try:
        spotify_df['lyrics'][i] = genius.search_song(spotify_df['track_name'][i], spotify_df['artist'][i]).lyrics
    except: 
        spotify_df['lyrics'][i] = '[Lyrics not found]'
        pass

In [ ]:
#Iterate through each row of the Spotify dataframe to search for lyrics based on the 'track_name' and 'artist' column
#Some songs may not find match to a lyrics page in the Genius website, in which case the except will populate message
for i in spotify_df[spotify_df['playlist_name'] == 'Electronic/Dance'].index:
    try:
        spotify_df['lyrics'][i] = genius.search_song(spotify_df['track_name'][i], spotify_df['artist'][i]).lyrics
    except: 
        spotify_df['lyrics'][i] = '[Lyrics not found]'
        pass

Running through each of the 5 API calls allowed us to successfully retrieve about 80% of the total songs in the dataset. With a considerable proportion of lyrics that were not successfully found using the wrapper, I explored the rows that were missing lyrics or had incorrect lyrics and discovered that this was likely due to certain songs having extended descriptions of the version of the song within the track name itself (ex. 'In Bloom - Nevermind Version'. The suffix of the song name where a hyphen was involved in the majority of the cases indicated a description of the song such as 'Live Version' or 'Album Version'. As a result, the lyrics genius API search was likely not finding the song since they stored the song simply by it's most recognized name (ie. In Bloom). In order to fix this, we updated the dataframe by removing the suffix of a track name wherever the character sequence ' - ' existed. Afterwords, we ran the API call once again.

In [ ]:
##Identify columns where lyrics were not initially found but contain a "-", indicating track name includes description
##such as 'Remastered' or 'Year Version' - update lyrics column to label and retrieve later
spotify_df.loc[(spotify_df['track_name'].str.contains(' - ')) \
& (spotify_df['lyrics'] == '[Lyrics not found]'), 'lyrics'] = 'Retry retrieving lyrics'

In [117]:
##Remove all text in track_name by splitting on " - "
spotify_df.loc[:, 'track_name'][spotify_df['lyrics'] == 'Retry retrieving lyrics'] = spotify_df.track_name.str.split(' - ', 1).str[0]

In [ ]:
#Retry retrieving lyrics on rows with updated track names
for i in spotify_df[spotify_df['lyrics'] == 'Retry retrieving lyrics'].index:
    try:
        spotify_df['lyrics'][i] = genius.search_song(spotify_df['track_name'][i], spotify_df['artist'][i]).lyrics
    except: 
        spotify_df['lyrics'][i] = '[Lyrics not found]'
        pass

VOILA! After running the call once again on the modified track names, we were able to retrieve the lyrics of about another 1000 songs, thus limiting the remaining missing lyrics to under 10% (about 900 songs). This is partially expected as a result of a good portion of songs either not having lyrics at all (ex. EDM songs being mostly instrumental) or obscure song and artist name combos which did not yield any results. With the lyrics sucessfully pulled into the dataframe, we will now need to tackle the challenge of doing some additional data wrangling and text pre-processing using various Natural Language Processing techniques. In order to ensure all of our lyrical data is saved and avoid potentially having to run 4-5 hours worth of queries again, we will save off our dataframe once again into a CSV and work with it in the next notebook.

In [1]:
#Read dataframe into CSV file to use in the next portion
spotify_df.to_csv("spotify_genre_df.csv", index=False)

NameError: name 'spotify_df' is not defined